# Day 6 of AoC 2018: Chronal Coordinates
https://adventofcode.com/2018/day/6

## Part one

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance

In [ ]:
points = pd.read_csv('day06_input.txt', names=['x', 'y'])

In [ ]:
len(points)

In [ ]:
points['label'] = range(len(points))

In [ ]:
labels = alt.Chart(points).mark_text(color='red').encode(
    x=alt.X('x:Q'),
    y=alt.Y('y:Q'),
    text=alt.Text('label:N')
)
labels

I will make a classifier with k nearest neighbors, using 1 neighbor and Manhattan distance. And then I will use this classifier to predict all points on a grid.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, metric="manhattan")

In [ ]:
knn.fit(points[['x', 'y']].values, np.arange(50))

I used [this answer from stackoverflow](https://stackoverflow.com/questions/32208359/is-there-a-multi-dimensional-version-of-arange-linspace-in-numpy) to create the grid

In [ ]:
x_min, x_max = points.x.min(), points.x.max()
y_min, y_max = points.y.min(), points.y.max()
grid = np.mgrid[x_min:(x_max+1), y_min:(y_max+1)].reshape(2,-1).T

In [ ]:
grid.shape

In [ ]:
predictions = knn.predict(grid)

In [ ]:
predictions_df = pd.DataFrame({'x':grid[:, 0],
                               'y':grid[:, 1],
                               'pred':predictions
                              })

In [ ]:
predictions_df.to_csv('day06_grid_predictions.csv', index=False)

In [ ]:
grid_chart = alt.Chart('day06_grid_predictions.csv').mark_point().encode(
    x=alt.X('x:Q'),
    y=alt.Y('y:Q'),
    color=alt.Color('pred:N', legend=None, scale=alt.Scale(scheme='category20')),
    tooltip='pred:N'
)

grid_chart + labels

Looks good! The area that seems to be overlapping are due to the fact that there are only 20 colors in the scheme, while I want to plot 50 zones, we can see that they are indeed separated with the tooltip.

I have to only take into account the finite areas, so any area that "touches" the ends has to be dismissed.

In [ ]:
def find_infinite_area(pred, num_labels=50, x_min=x_min, 
                       x_max=x_max, y_min=y_min, y_max=y_max):
    """
    Returns a set of labels the touches the ends
    """
    infinite_area = set()
    x_min
    for label in range(num_labels):
        label_df = pred.query('pred == @label')
        if (x_min in label_df['x'].values) or (x_max in label_df['x'].values)\
          or (y_min in label_df['y'].values) or (y_max in label_df['y'].values):
            infinite_area.add(label)
    return infinite_area

In [ ]:
infinite_area = find_infinite_area(predictions_df)
print(infinite_area)

I update the dataframe

In [ ]:
predictions_df['infinite'] = predictions_df['pred'].isin(infinite_area)

In [ ]:
predictions_df.to_csv('day06_grid_predictions.csv', index=False)

In [ ]:
grid_chart.encode(color=alt.Color('infinite:O')) + labels

OK, it seems to have done a good job! Let's now filter those areas and count the number of values, which will give the total area. I will have to take care of points being at an equal distance of 2 data points.

In [ ]:
predictions_df.query('not infinite')['pred'].value_counts()[:5]

So, the area 10 is the largest. The value does not take into account that I should not count the points that are at the same distance of 2 labels.

In [ ]:
area10 = predictions_df.query('pred == 10').loc[:, ['x', 'y']].values

In [ ]:
def center_coord(id):
    return points.loc[id, ['x', 'y']].values.reshape(-1, 2)


def matrix_neighbor(ids=None):
    if ids is None:
        # This is from visual inspection
        ids = [33, 20, 22, 42, 46]
    all_neighbors = np.zeros(shape=(len(ids), 2))
    for i, id in enumerate(ids):
        all_neighbors[i, :] = center_coord(id)
    return all_neighbors

In [ ]:
coords_neighbors_10 = matrix_neighbor()

In [ ]:
distances_neighbors = distance.cdist(area10, coords_neighbors_10, metric='cityblock')

In [ ]:
distances_10 = distance.cdist(area10, center_coord(10), metric='cityblock')

In [ ]:
differences = distances_neighbors - distances_10

In [ ]:
np.sum(np.any(differences == 0, axis=1))

There are 66 ties in the distances, so we must substract those from the area.

In [ ]:
print(f'Solution for part 1: {len(area10) - 66}')

## Part 2

Let's compute the distance for our grid of points to all the points.

In [ ]:
all_distances = distance.cdist(grid, points.loc[:, ['x', 'y']].values, metric='cityblock')

Now, we need to sum all the distances.

In [ ]:
sum_distances = np.sum(all_distances, axis=1)

We need to find and count the values that are less than 10000.

In [ ]:
print(f'Solution for part 2: {np.sum(sum_distances < 10000)}')

Just for fun, plot this

In [ ]:
predictions_df['sum_dist'] = sum_distances
predictions_df['close'] = sum_distances < 10000

In [ ]:
predictions_df.to_csv('day06_grid_predictions.csv', index=False)

In [ ]:
grid_chart.encode(color=alt.Color('sum_dist:Q', bin=True),
                  tooltip=['sum_dist:Q', 'close:O']) + labels.mark_point(color='red')